In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
from tqdm import tqdm_notebook

In [2]:
url = 'https://www.reclameaqui.com.br/'

In [3]:
def get_soup(url,xpath):
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    try:
        element = driver.find_element_by_xpath(xpath)
        html_content = element.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content,'html.parser')
        return soup
    except:
        pass

In [5]:
##Scraping all the categories from the website

data = {}
soup = get_soup(url+'/categoria','//*[@id="all-categories"]/div/div')
for category in soup.findAll('a',{'class':'hidden-xs'}):
    data[category.find('div').h2.contents[0]] = {'category_link':category.get('href'),'companies_links':{}}

In [10]:
#getting all companies from categories rankings

for k,v in tqdm_notebook(data.items()):
    content = get_soup(url+v['category_link'],'//*[@id="category-rankings"]')
    for c in content.findAll('div',{'class':'box-gray'}):
        data[k]['companies_links'].update({c.h3.text:[]})
        for link in c.findAll('a',{'class':'business-name ng-binding'}):
            data[k]['companies_links'][c.h3.text].append(link.get('href'))


In [12]:
#Save json
with open('./exports/company_urls', 'w') as f:
    json.dump(data,f)

In [53]:
#Save csv
consolidated_data = []
for category in data:
    for sub in data[category]['companies_links']:
        for url in data[category]['companies_links'][sub]:
            consolidated_data.append((category,sub,url))
df = pd.DataFrame(consolidated_data, columns=['category','sub_category','company_url'])
df.to_csv('./exports/consolidate_company_urls.csv',index=False)

In [40]:
import pandas as pd
import requests
import time

In [9]:
page = 10
company_id = ''
company_url = ''
company_id_url = 'https://iosite.reclameaqui.com.br/raichu-io-site-v1/company/shortname/'+company_url

In [10]:
df = pd.read_csv('./exports/consolidate_company_urls.csv')

In [28]:
company_url = df['company_url'].loc[500].split('/empresa/')[1]

In [36]:
r = requests.get(company_id_url+company_url)
r.status_code
company_id = r.json()['id']

In [42]:
time.sleep(2)
comments_url = 'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/'+str(page)+'?company='+company_id+'&evaluated=bool:true'
r = requests.get(comments_url)
r.status_code

200

In [50]:
max_page = r.json()['complainResult']['complains']['count']
max_page

1314

In [52]:
comments = r.json()['complainResult']['complains']['data']
comments

[{'additionalFields': None,
  'moderationUserName': '****',
  'companyName': 'Anatel - Agência Nacional de Telecomunicações',
  'otherProblemType': 'Aparentemente não regula',
  'solved': False,
  'dealAgain': False,
  'otherProductType': 'Regular',
  'type': None,
  'evaluation': 'não houve',
  'score': 0.0,
  'userState': 'CE',
  'userRequestedDelete': False,
  'requesterName': None,
  'additionalInfo': '',
  'modified': '2020-01-02T16:13:22',
  'id': 'bA55HFLjK69_mVXI',
  'evaluated': True,
  'presence': None,
  'publishedEmailSent': None,
  'deletionReason': '****',
  'read': False,
  'marketplaceComplain': None,
  'created': '2019-12-23T14:46:44',
  'ip': '****',
  'moderationReasonDescription': '****',
  'frozen': False,
  'requestEvaluation': None,
  'moderateReason': None,
  'files': None,
  'problemType': '0000000000000000',
  'deletedIp': '****',
  'companyIndexes': ['{averageAnswerTime=0.0, finalScore=1.8, totalNotAnswered=454, totalComplains=454, consumerScore=1.75, created